In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
import tarfile
import pathlib
import pickle
import numpy as np
from spladder.viz.graph import *
import h5py

Matplotlib created a temporary config/cache directory at /scratch/slurm-job.1088651/matplotlib-314s9xxm because the default path (/cluster/customapps/biomed/grlab/users/prelotla/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
# Inputs ETH
run_type = 'ov'

if run_type == 'brca':
   
    sample_target = 'TCGA-AO-A0JM-01A-21R-A056-07'
    basedir = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102'
    filtering_id = 'filters_22March_order_wany_wAnnot' # OLD!!!!
elif run_type == 'ov':
    sample_target = 'TCGA-25-1319-01A-01R-1565-13'
    basedir = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374'
    filtering_id = 'filters_22March_order_wany_wAnnot'

filter_dir = os.path.join(basedir, 'filtering_samples', filtering_id)

# File obtained from 
# 1. Filtered kmer list 
# 2. Generation info
# 3. Added a lot of false positive related information in projects2020_ohsu/eth/debug/debug_false_positives/p20230307_filter_allRF_quality_check.ipynb

metadata = 'metadata2'
path_interest = f'G_{sample_target}_SampleLim0.0CohortLimNoneAcrossNone_FiltNormalsGtexCohortCohortlim0.0Across1{metadata}.tsv.gz'
path = os.path.join(filter_dir, path_interest)


# Inputs OHSU
path_ohsu = f'output/kmer_files/J_{sample_target}_SampleLim0.0CohortLimNoneAcrossNone_FiltNormalsGtexCohortCohortlim0.0Across1.tsv'
tar_file_OHSU = '/cluster/work/grlab/projects/projects2020_OHSU/share_OHUS_PNLL/include_ANY_filters_OHSU-results_March22-2023/OHSU_March27-2023_fixAnyfilters_results.tar.gz'

# Spladder 
splice_path = '/cluster/work/grlab/projects/GTEx/rna_gencode32_realign/results/splicing/spladder/genes_graph_conf2.merge_graphs.pickle'
#splice_path = '/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Breast_1102_results/splicing/spladder/genes_graph_conf2.merge_graphs.pickle'
count_path = '/cluster/work/grlab/projects/GTEx/rna_gencode32_realign/results/splicing/spladder/genes_graph_conf2.merge_graphs.count.rechunked.hdf5'


# Get batch equivalence
batch_gtex = pd.read_csv('/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/GTEX2019_eth/GTEX2019_c4dd02c_conf2_RFall_ref/batch_to_gene.txt', names = ['gene', 'batch_gtex'])
if run_type == 'brca':
    batch_cancer = pd.read_csv('/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102/batch_to_gene.txt', names = ['gene', 'batch_cancer'])
elif run_type == 'ov':
    batch_cancer = pd.read_csv('/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/batch_to_gene.txt', names = ['gene', 'batch_cancer'])
batch_table = batch_cancer.merge(batch_gtex, on = 'gene', how = 'outer')

In [3]:
batch_table.head()

,gene,batch_cancer,batch_gtex
0,ENSG00000169962.5,100,100
1,ENSG00000062282.15,10000,10000
2,ENSG00000198382.9,10004,10004
3,ENSG00000085741.13,10012,10012
4,ENSG00000137492.8,10015,10015


In [4]:
# Read ETH Data
df = pd.read_csv(path, sep = '\t')
print(f'Reading {path}')
display(df.head())


# Read OHSU Data
with tarfile.open(tar_file_OHSU, "r:*") as tar: #OHSU
    df_ohsu = pd.read_csv(tar.extractfile(path_ohsu), header=0, sep=",")
    
# Load Spladder
with open(splice_path, 'rb') as graph_fp:
    (graph_data, graph_meta) = pickle.load(graph_fp, encoding='latin1')  # both graph data and meta data

Reading /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_samples/filters_22March_order_wany_wAnnot/G_TCGA-25-1319-01A-01R-1565-13_SampleLim0.0CohortLimNoneAcrossNone_FiltNormalsGtexCohortCohortlim0.0Across1metadata2.tsv.gz


,kmer,gtexCohortfilter >0.0,gtexCohortfilter >=1.0,gtexCohortfilter >=2.0,gtexCohortfilter >=3.0,gtexCohortfilter >=5.0,gtexCohortfilter >=10.0,coord,junctionAnnotated,readFrameAnnotated,...,TCGA61200801A02R156813all,TCGA24143101A01R156613all,TCGA24229801A01R156913all,isAnnotated,strand,junction_coordinate,coordinate_in_GTEX,gtexExon1<cancExon1,gtexExon2<cancExon2,info_not_available
0,RMREATKGP,NaN,NaN,NaN,NaN,NaN,NaN,55350885:55350897:55352540:55352555:None:None,False,True,...,0.0,0.0,0.0,NaN,+,55350897:55352540,False,False,False,False
1,QVERMREAT,NaN,NaN,NaN,NaN,NaN,NaN,55350876:55350897:55352540:55352546:None:None,False,True,...,0.0,0.0,0.0,NaN,+,55350897:55352540,False,False,False,False
2,VERMREATK,NaN,NaN,NaN,NaN,NaN,NaN,55350879:55350897:55352540:55352549:None:None,False,True,...,0.0,0.0,0.0,NaN,+,55350897:55352540,False,False,False,False
3,MREATKGPV,NaN,NaN,NaN,NaN,NaN,NaN,55350888:55350897:55352540:55352558:None:None,False,True,...,0.0,0.0,0.0,NaN,+,55350897:55352540,False,False,False,False
4,VQVERMREA,NaN,NaN,NaN,NaN,NaN,NaN,55350873:55350897:55352540:55352543:None:None,False,True,...,0.0,0.0,0.0,NaN,+,55350897:55352540,False,False,False,False


In [5]:
# Data Wrangling
recurr_col = 'cancerCohortfilter >0.0'
expr_col = sample_target.replace('-', '') + 'all'
check_in_gtex_coord = 'coordinate_in_GTEX'
check_exons_length = 'gtexExon2<cancExon2'
jxannot = 'junctionAnnotated'
rfannot = 'readFrameAnnotated'
idx = 'index'
info = 'info_not_available'
df = df.sort_values(by=[recurr_col])
df[idx] = np.arange(len(df))

# Compute shared kmers across the two pipelines
shared_col = 'shared_across_labs'
shared_kmers = set(df_ohsu['kmer']).intersection(set(df['kmer']))

df_ohsu = df_ohsu.set_index('kmer')
df_ohsu.loc[list(shared_kmers), shared_col] = True
df_ohsu = df_ohsu.reset_index().drop_duplicates()

# Merge to ETH
df = df.merge(df_ohsu, on = 'kmer', how = 'left')
df[shared_col] = df[shared_col].fillna(False)

In [6]:
# Test
foo = df[['kmer', recurr_col, expr_col, check_in_gtex_coord, check_exons_length, jxannot, rfannot, info, idx, 'junction_coordinate', 'batch' , 'shared_across_labs']]

foo.loc[(foo['cancerCohortfilter >0.0'] >=100) & (foo['shared_across_labs'] == False)][0:20]

,kmer,cancerCohortfilter >0.0,TCGA25131901A01R156513all,coordinate_in_GTEX,gtexExon2<cancExon2,junctionAnnotated,readFrameAnnotated,info_not_available,index,junction_coordinate,batch,shared_across_labs
108706,IACWRPSLP,100,6.714437,False,False,False,True,False,108706,53873969:53874060,29614,False
108707,ELARKNRDV,100,0.839305,False,False,False,False,False,108707,52951613:52951804,12423,False
108708,LGRTETYSP,100,0.839305,False,False,False,False,False,108708,52951613:52951804,12423,False
108709,VDPTPAAAT,100,1.678609,False,False,False,False,False,108709,76304002:76304189,51201,False
108710,DELARKNRD,100,0.839305,False,False,False,False,False,108710,52951613:52951804,12423,False
108711,FFSCIACWR,100,6.714437,False,False,False,True,False,108711,53873969:53874060,29614,False
108712,PRCGPHPCC,100,1.678609,False,False,False,True,False,108712,76304002:76304189,51201,False
108714,TFFSCIACW,100,6.714437,False,False,False,True,False,108714,53873969:53874060,29614,False
108715,NRDVQSSPW,100,0.839305,False,False,False,False,False,108715,52951613:52951804,12423,False
108717,FSCIACWRP,100,6.714437,False,False,False,True,False,108717,53873969:53874060,29614,False


In [7]:
coord_explore = df[['batch', 'junction_coordinate']].drop_duplicates()

# Extract Coordinates from spladder 
spladder_info = []
number_samples_expr = []
junction_coord = []
# -45 min 
for row_idx, row in coord_explore.iterrows():
    
    ### Extract characteristics from the candidate 
    if len(row['junction_coordinate'].split(':')) == 2: 
        pattern_right, pattern_left = np.sort(np.array(row['junction_coordinate'].split(':'),  dtype=int))
        gene_id = row['batch']
        gene_id = batch_table.loc[batch_table['batch_cancer'] == gene_id, 'batch_gtex'].values[0]
        
        ### Get Gene 
        gene = graph_data[gene_id]

        ### GET THE Number of the vertexes
        graph_edge_idx = None
        first = np.where(gene.segmentgraph.seg_edges)[0]
        second = np.where(gene.segmentgraph.seg_edges)[1]
        for edge_idx in np.arange(len(first)):
            v1 = first[edge_idx]
            v2 = second[edge_idx]
            # Same - and + strand
            if (str(pattern_left) in  str(gene.segmentgraph.segments[0][v2])) and \
                (str(pattern_right) in str(gene.segmentgraph.segments[1][v1])):
                graph_edge_idx = edge_idx

        if graph_edge_idx is not None:
            # Get edge h5py indexes
            with h5py.File(count_path, 'r') as h5f:  
                test_edge_id = [i for i, j  in enumerate(h5f['gene_ids_edges'])  if j == gene_id]
            h5_edge = test_edge_id[graph_edge_idx]

            # Get expression h5py
            with h5py.File(count_path, 'r') as h5f: test_edg_exr=h5f['edges'][h5_edge, :]


            spladder_info.append(f'SGv1-{v1};SGv2-{v2};SGe-{graph_edge_idx};h5e-{h5_edge}')
            number_samples_expr.append(len(test_edg_exr[np.where(test_edg_exr > 0)[0]]))
            junction_coord.append(row['junction_coordinate'])



Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 25226 25225
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential issue 7829 7828
Potential 

In [15]:
# Merge with pevious data
coord_explore_info = pd.DataFrame({'spladder_info': spladder_info, 
                                   'number_samples_expr': number_samples_expr,
                                   'junction_coordinate': junction_coord})

df = df.merge(coord_explore_info, on = 'junction_coordinate', how = 'left')

In [10]:
coord_explore_info.shape


(88, 3)

In [11]:
coord_explore_info.loc[coord_explore_info['number_samples_expr'] > 0, 'number_samples_expr'].describe()

count      83.000000
mean     1163.759036
std      2487.497394
min         1.000000
25%        12.500000
50%        55.000000
75%       524.500000
max      9618.000000
Name: number_samples_expr, dtype: float64

In [12]:
coord_explore.shape

(16761, 2)

In [13]:
# Save intermediate data
metadata = 'metadata-spladder'
path_interest = f'G_{sample_target}_SampleLim0.0CohortLimNoneAcrossNone_FiltNormalsGtexCohortCohortlim0.0Across1{metadata}.tsv.gz'
path = os.path.join(filter_dir, path_interest)


df.to_csv(path, compression = 'gzip', index = None)

In [14]:
path


'/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_samples/filters_22March_order_wany_wAnnot/G_TCGA-25-1319-01A-01R-1565-13_SampleLim0.0CohortLimNoneAcrossNone_FiltNormalsGtexCohortCohortlim0.0Across1metadata-spladder.tsv.gz'

# Interpretations

#### Numbers

In [17]:
metadata = 'metadata-spladder'
path_interest = f'G_{sample_target}_SampleLim0.0CohortLimNoneAcrossNone_FiltNormalsGtexCohortCohortlim0.0Across1{metadata}.tsv.gz'
path = os.path.join(filter_dir, path_interest)

df = pd.read_csv(path)

In [18]:
df.head()

,kmer,gtexCohortfilter >0.0,gtexCohortfilter >=1.0,gtexCohortfilter >=2.0,gtexCohortfilter >=3.0,gtexCohortfilter >=5.0,gtexCohortfilter >=10.0,coord,junctionAnnotated,readFrameAnnotated,...,strand,junction_coordinate,coordinate_in_GTEX,gtexExon1<cancExon1,gtexExon2<cancExon2,info_not_available,index,shared_across_labs,spladder_info,number_samples_expr
0,LKFLYSRPI,NaN,NaN,NaN,NaN,NaN,NaN,1997343:1997348:1993163:1993176:1993078:1993087,True,True,...,-,1993176:1997343:1993163:1993087,False,False,False,False,0,True,NaN,NaN
1,NGLLSTLQK,NaN,NaN,NaN,NaN,NaN,NaN,102905319:102905325:102908697:102908718:None:None,False,False,...,+,102905325:102908697,False,False,False,False,1,True,NaN,NaN
2,TASYNGLLS,NaN,NaN,NaN,NaN,NaN,NaN,102905307:102905325:102908697:102908706:None:None,False,False,...,+,102905325:102908697,False,False,False,False,2,True,NaN,NaN
3,TLMAQMASG,NaN,NaN,NaN,NaN,NaN,NaN,102044321:102044332:102044477:102044493:None:None,False,False,...,+,102044332:102044477,not_available,not_available,not_available,True,3,True,NaN,NaN
4,ASYNGLLST,NaN,NaN,NaN,NaN,NaN,NaN,102905310:102905325:102908697:102908709:None:None,False,False,...,+,102905325:102908697,False,False,False,False,4,True,NaN,NaN


In [20]:
print(len(df))

131962


In [21]:
print(len(df['kmer'].unique()))

113574


In [22]:
print(len(df['junction_coordinate'].unique()))

16755


In [23]:
print(len(df.loc[df['shared_across_labs'] == True, :]))

43156


In [24]:
print(len(df.loc[df['shared_across_labs'] == True, 'kmer'].unique()))

36885


In [25]:
print(len(df.loc[df['shared_across_labs'] == True, 'junction_coordinate'].unique()))

7888


In [84]:
df.groupby('coordinate_in_GTEX').count() # From Metadata 

,kmer,gtexCohortfilter >0.0,gtexCohortfilter >=1.0,gtexCohortfilter >=2.0,gtexCohortfilter >=3.0,gtexCohortfilter >=5.0,gtexCohortfilter >=10.0,coord,junctionAnnotated,readFrameAnnotated,...,isAnnotated,strand,junction_coordinate,gtexExon1<cancExon1,gtexExon2<cancExon2,info_not_available,index,shared_across_labs,spladder_info,number_samples_expr
coordinate_in_GTEX,,,,,,,,,,,,,,,,,,,,,
False,125496,183,183,183,183,183,183,125496,125496,125496,...,6,125496,125496,125496,125496,125496,125496,125496,4,4
True,848,96,96,96,96,96,96,848,848,848,...,8,848,848,848,848,848,848,848,344,344
not_available,5618,404,404,404,404,404,404,5618,5618,5618,...,8,5618,5618,5618,5618,5618,5618,5618,66,66


In [86]:
print(len(df.loc[df['coordinate_in_GTEX'] == 'True', 'junction_coordinate'].unique()))  # From Metadata 

253


In [117]:
# Remark discrepency comes from 3-exons which seem to be found in the metadata. But not in the GTEX graph
display(df.loc[df['coordinate_in_GTEX'] == 'True', ['junction_coordinate', 'number_samples_expr']].drop_duplicates().head())

,junction_coordinate,number_samples_expr
2299,6857723:6860369,0.0
5034,53818013:53831314,51.0
6387,13109821:13110157,0.0
8002,67483174:67483242:67483257:67487005,NaN
8012,67483174:67483242:67483257:67490037,NaN


In [118]:
# Print 1 kmer per case
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    display(df.loc[df['coordinate_in_GTEX'] == 'True', ['junction_coordinate', 'number_samples_expr']].drop_duplicates())

,junction_coordinate,number_samples_expr
2299,6857723:6860369,0.0
5034,53818013:53831314,51.0
6387,13109821:13110157,0.0
8002,67483174:67483242:67483257:67487005,NaN
8012,67483174:67483242:67483257:67490037,NaN
13094,74762173:74768149,0.0
13721,82884202:82889679,27.0
14965,72275636:72275657,13.0
27228,81904479:81904687,1.0
33505,33199416:33200241:33199393:33199319,NaN


In [119]:
#df_test = df[ df['coordinate_in_GTEX'] == 'True']  # From Graph 
df_test = df[ ~ df['number_samples_expr'].isna()]  # From Graph 
print(len(df_test['junction_coordinate'].unique()))

88


In [120]:
print(len(df_test.loc[(df['number_samples_expr'] > 0), 'junction_coordinate'].unique()))


83


In [121]:
print(len(df_test.loc[(df_test['number_samples_expr'] > 0) & 
            (df_test['gtexExon1<cancExon1'] == 'True' ), 'junction_coordinate'].unique()))

34


In [122]:
print(len(df_test.loc[(df_test['number_samples_expr'] > 0) & 
            (df_test['gtexExon2<cancExon2'] == 'True' ), 'junction_coordinate'].unique()))

40


In [123]:
print(len(df_test.loc[ \
            (df_test['number_samples_expr'] > 0) & 
            (df_test['gtexExon2<cancExon2'] != 'True' ) &
            (df_test['gtexExon1<cancExon1'] != 'True' ), 'junction_coordinate'].unique()))

14


In [124]:
df_test.loc[ \
            (df_test['number_samples_expr'] > 0) & 
            (df_test['gtexExon2<cancExon2'] != 'True' ) &
            (df_test['gtexExon1<cancExon1'] != 'True' ), 'batch'].unique()

array([ 9852, 25656, 48360, 21278,  6250, 17567,   160,  5518, 14553,
        6782])

In [126]:
df_test.loc[ \
            (df_test['number_samples_expr'] > 0) & 
            (df_test['gtexExon2<cancExon2'] != 'True' ) &
            (df_test['gtexExon1<cancExon1'] != 'True' ), 'gtexExon1<cancExon1'].unique()

array(['not_available', 'False'], dtype=object)

In [127]:
df_test.loc[ \
            (df_test['number_samples_expr'] > 0) & 
            (df_test['gtexExon2<cancExon2'] != 'True' ) &
            (df_test['gtexExon1<cancExon1'] != 'True' ), 'gtexExon2<cancExon2'].unique()

array(['not_available', 'False'], dtype=object)

In [128]:
print(len(df_test.loc[ \
            (df_test['number_samples_expr'] > 0) & 
            (df_test['gtexExon2<cancExon2'] == 'False' ) &
            (df_test['gtexExon1<cancExon1'] == 'False' ), 'junction_coordinate'].unique()))             

2


In [130]:
df_test.loc[ \
            (df_test['number_samples_expr'] > 0) & 
            (df_test['gtexExon2<cancExon2'] == 'False' ) &
            (df_test['gtexExon1<cancExon1'] == 'False' ), 'batch'].unique()

array([25656, 21278])

In [129]:
print(len(df_test.loc[ \
            (df_test['number_samples_expr'] > 0) & 
            (df_test['gtexExon2<cancExon2'] == 'not_available' ) &
            (df_test['gtexExon1<cancExon1'] == 'not_available' ), 'junction_coordinate'].unique()))       

12


In [134]:
print(len(df_test.loc[ \
            (df_test['number_samples_expr'] > 0) & 
            (df_test['gtexExon2<cancExon2'] == 'not_available' ) &
            (df_test['gtexExon1<cancExon1'] == 'not_available' ), 'batch'].unique()))

8


In [135]:
df_test.loc[ \
            (df_test['number_samples_expr'] > 0) & 
            (df_test['gtexExon2<cancExon2'] == 'not_available' ) &
            (df_test['gtexExon1<cancExon1'] == 'not_available' ), 'batch'].unique()
#NOTE ACTUALLY THE INFORMATION IS AVAILABLE BUT IT IS SPLIT IN SEVERAL FILES + There are kmers missing = incomplete runs

array([ 9852, 48360,  6250, 17567,   160,  5518, 14553,  6782])

#### kmers

In [64]:
cols_interest =  ['kmer', recurr_col, expr_col, 
               check_in_gtex_coord, check_exons_length, 
               jxannot, rfannot, info, idx, 
               'junction_coordinate', 'batch' , 'shared_across_labs']

In [65]:
## Case True positive # Junction is present in spladder but maybe some other good reason not to translate it 
df_test.loc[ \
            (df_test['number_samples_expr'] > 0) & 
            (df_test['gtexExon2<cancExon2'] == 'False' ) &
            (df_test['gtexExon1<cancExon1'] == 'False' ), cols_interest]

,kmer,cancerCohortfilter >0.0,TCGA25131901A01R156513all,coordinate_in_GTEX,gtexExon2<cancExon2,junctionAnnotated,readFrameAnnotated,info_not_available,index,junction_coordinate,batch,shared_across_labs
27228,AYKVQSRGL,9,6.714437,True,False,False,False,False,27228,81904479:81904687,25656,False
27229,LPAYKVQSR,9,6.714437,True,False,False,False,False,27229,81904479:81904687,25656,False
27230,ASLPAYKVQ,9,6.714437,True,False,False,False,False,27230,81904479:81904687,25656,False
27231,SLPAYKVQS,9,6.714437,True,False,False,False,False,27231,81904479:81904687,25656,False
27232,PAYKVQSRG,9,6.714437,True,False,False,False,False,27232,81904479:81904687,25656,False
103713,GAHQLPPGG,86,0.839305,True,False,False,False,False,103713,31132842:31132904,21278,True
103719,KLGAHQLPP,86,0.839305,True,False,False,False,False,103719,31132842:31132904,21278,True
103722,PTSYLQEGS,86,0.839305,True,False,False,False,False,103722,31132842:31132904,21278,False
103739,WGPTSYLQE,86,0.839305,True,False,False,False,False,103739,31132842:31132904,21278,False
103741,SSWGPTSYL,86,0.839305,True,False,False,False,False,103741,31132842:31132904,21278,False


In [69]:
## Case metadata not available (Cannot check quickly the exon length)
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    display(df_test.loc[ \
                (df_test['number_samples_expr'] > 0) & 
                (df_test['gtexExon2<cancExon2'] == 'not_available' ) &
                (df_test['gtexExon1<cancExon1'] == 'not_available' ), cols_interest])

,kmer,cancerCohortfilter >0.0,TCGA25131901A01R156513all,coordinate_in_GTEX,gtexExon2<cancExon2,junctionAnnotated,readFrameAnnotated,info_not_available,index,junction_coordinate,batch,shared_across_labs
17788,KLEALRAER,6,7.554,not_available,not_available,False,False,True,17788,72013017:72014118,9852,True
17812,AEREAQSTA,6,7.554,not_available,not_available,False,False,True,17812,72013017:72014118,9852,True
18022,LRAEREAQS,6,7.554,not_available,not_available,False,False,True,18022,72013017:72014118,9852,True
18474,ALRAEREAQ,6,7.554,not_available,not_available,False,False,True,18474,72013017:72014118,9852,True
18499,LEALRAERE,6,7.554,not_available,not_available,False,False,True,18499,72013017:72014118,9852,True
18539,RAEREAQST,6,7.554,not_available,not_available,False,False,True,18539,72013017:72014118,9852,True
18581,EREAQSTAR,6,7.554,not_available,not_available,False,False,True,18581,72013017:72014118,9852,True
19762,EALRAEREA,6,7.554,not_available,not_available,False,False,True,19762,72013017:72014118,9852,True
59539,TAKMIHLWV,25,0.839,not_available,not_available,False,False,True,59539,73518040:73518387,48360,False
107300,IASGTTLLE,95,0.839,not_available,not_available,False,False,True,107300,73518802:73519104,48360,False


In [72]:
## Case metadata not available (Cannot check quickly the exon length)
display(df_test.loc[ \
                (df_test['number_samples_expr'] > 0) & 
                (df_test['gtexExon2<cancExon2'] == 'not_available' ) &
                (df_test['gtexExon1<cancExon1'] == 'not_available' ), 'batch'].unique())

array([ 9852, 48360,  6250, 17567,   160,  5518, 14553,  6782])

foo = set([ 9852, 48360,  6250, 17567,   160,  5518, 14553,  6782]) 
bar = set ([14553, 160,17567, 5518, 6250, 9852, 21278, 25656 ])

Case metadata not available (Cannot check quickly the exon length)

array([ 9852, 48360,  6250, 17567,   160,  5518, 14553,  6782])

Compare with non completed batches 
batch 14553 - YES (non completed, and metadata info not available)
batch 160 - YES
batch 17567 - YES
batch 5518 - YES
batch 6250 - YES 
batch 9852 - YES 
batch 21278 - PROBABLY NON COMPLETED. NO OVERLAP REAL ISSUE
batch 25656 - PROBABLY NON COMPLETED. NO OVERLAP REAL ISSUE

48360, 6782 (metadata info not available)

21278, 25656 (True positives, make a rerun to confirm)

In [74]:
# # Print 1 kmer per case
# with pd.option_context('display.max_rows', None,
#                        'display.max_columns', None,
#                        'display.precision', 3,
#                        ):
#     display(df_test.loc[(df['gtexExon2<cancExon2'] == False) & (df['number_samples_expr'] > 0),
#               ['kmer', recurr_col, expr_col, 
#                check_in_gtex_coord, check_exons_length, 
#                jxannot, rfannot, info, idx, 
#                'junction_coordinate', 'batch' , 'shared_across_labs']].drop_duplicates(subset='junction_coordinate', \
#                                                                                        keep = 'last'))


In [7]:
# # Save intermediate data
# metadata = 'metadata-spladder-pb'
# path_interest = f'G_{sample_target}_SampleLim0.0CohortLimNoneAcrossNone_FiltNormalsGtexCohortCohortlim0.0Across1{metadata}.tsv.gz'
# path = os.path.join(filter_dir, path_interest)
# df_test = df_test.merge(batch_table, left_on = 'batch', right_on = 'batch_cancer')
# df_test.to_csv(path, sep = ',')

# #df_test.loc[(df['gtexExon2<cancExon2'] == False) & (df['number_samples_expr'] > 0), :].to_csv(path, compression = 'gzip', index = None)

In [61]:
print(path)

/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_samples/filters_22March_order_wany_wAnnot/G_TCGA-25-1319-01A-01R-1565-13_SampleLim0.0CohortLimNoneAcrossNone_FiltNormalsGtexCohortCohortlim0.0Across1metadata-spladder.tsv.gz
